# Churn Case Part 2: Analysis Of LTV (Life Time Value)

Lifetime value (LTV) is the total worth to a business of a customer over the while period of their relationship. It's an important metric as it costs less to keep existings customers than to acquire new ones, so the idea of increasing the value of your existig customers is a great way to drive growth.

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("churn.db")

In [3]:
df = pd.read_sql("select * from churn_all", conn)

In [4]:
df.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.5,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,Yes,...,Yes,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,3046.05,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,Yes,...,No,Yes,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.3,Yes


## 1. For those customers who unsubscribed the service, what was their average LTV? and how long did they usually stay in the service?

In [5]:
## Extract those who unsubscribed the serviced (Churn = Yes)

churn_df = pd.read_sql("select * from churn_all where Churn = 'Yes'", conn)

In [6]:
# Examine the distribution of TotalCharges
churn_df['TotalCharges'] = churn_df['TotalCharges'].astype(float)
churn_df["TotalCharges"].describe()

count    1869.000000
mean     1531.796094
std      1890.822994
min        18.850000
25%       134.500000
50%       703.550000
75%      2331.300000
max      8684.800000
Name: TotalCharges, dtype: float64

We can see that around 20% of the TotalCharges are very high, so let's divide the data to see each distribution.

In [7]:
# Find the 80th percentile of the data in TotalCharges
churn_df.TotalCharges.quantile(0.8)

2840.4100000000003

In [8]:
pd.read_sql("select * from churn_all where TotalCharges <= 2840.41", conn)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,1066-JKSGK,Male,No,No,No,California,34.066303,-118.435479,90024,Yes,...,No internet service,No internet service,No internet service,1,Month-to-month,Mailed check,No,20.15,20.15,Yes
3,8773-HHUOZ,Female,No,No,No,California,34.078821,-118.177576,90032,Yes,...,No,Yes,Yes,17,Month-to-month,Mailed check,Yes,64.70,1093.1,Yes
4,8168-UQWWF,Female,No,No,Yes,California,34.115720,-118.192754,90042,Yes,...,No,Yes,Yes,11,Month-to-month,Bank transfer (automatic),Yes,97.85,1105.4,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2696,2235-DWLJU,Female,Yes,No,No,California,33.849263,-116.382778,92276,No,...,No,Yes,Yes,6,Month-to-month,Electronic check,Yes,44.40,263.05,No
2697,9767-FFLEM,Male,No,No,No,California,33.036790,-115.605030,92281,Yes,...,No,No,No,38,Month-to-month,Credit card (automatic),Yes,69.50,2625.25,No
2698,8456-QDAVC,Male,No,No,No,California,32.852947,-114.850784,92283,Yes,...,No,Yes,No,19,Month-to-month,Bank transfer (automatic),Yes,78.70,1495.1,No
2699,2569-WGERO,Female,No,No,No,California,34.341737,-116.539416,92285,Yes,...,No internet service,No internet service,No internet service,72,Two year,Bank transfer (automatic),Yes,21.15,1419.4,No


In [9]:
churn_df.query("TotalCharges <= 2840.41")

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.50,Yes
5,4190-MFLUW,Female,No,Yes,No,California,34.066367,-118.309868,90020,Yes,...,Yes,No,No,10,Month-to-month,Credit card (automatic),No,55.20,528.35,Yes
6,8779-QRDMV,Male,Yes,No,No,California,34.023810,-118.156582,90022,No,...,No,No,Yes,1,Month-to-month,Electronic check,Yes,39.65,39.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863,1122-JWTJW,Male,No,Yes,Yes,California,32.698964,-115.886656,92259,Yes,...,No,No,No,1,Month-to-month,Mailed check,Yes,70.65,70.65,Yes
1864,1699-HPSBG,Male,No,No,No,California,33.745746,-116.514215,92264,Yes,...,Yes,Yes,No,12,One year,Electronic check,Yes,59.80,727.80,Yes
1865,8775-CEBBJ,Female,No,No,No,California,32.790282,-115.689559,92273,Yes,...,No,No,No,9,Month-to-month,Bank transfer (automatic),Yes,44.20,403.35,Yes
1866,6894-LFHLY,Male,Yes,No,No,California,34.264124,-114.717964,92280,Yes,...,No,No,No,1,Month-to-month,Electronic check,Yes,75.75,75.75,Yes


In [10]:
# Divide the data by the 80th percentile of the data from the TotalCharges variable
total_charges_under80 = churn_df.query("TotalCharges <= 2840.41")
total_charges_above80 = churn_df.query("TotalCharges > 2840.41")

In [11]:
# Show the distribution of people under the 80th percentile
total_charges_under80.TotalCharges.describe()

count    1495.000000
mean      711.265819
std       766.848197
min        18.850000
25%        85.025000
50%       371.650000
75%      1128.225000
max      2839.650000
Name: TotalCharges, dtype: float64

In [12]:
# Show the distribution of people above the 80th percentile
total_charges_above80.TotalCharges.describe()

count     374.000000
mean     4811.723262
std      1436.724288
min      2841.550000
25%      3522.462500
50%      4571.100000
75%      5891.212500
max      8684.800000
Name: TotalCharges, dtype: float64

In [13]:
# Show the distribution of the tenure of people under the 80th percentile
total_charges_under80.Tenure.describe()

count    1495.000000
mean        9.935117
std        10.742349
min         1.000000
25%         1.000000
50%         6.000000
75%        15.000000
max        61.000000
Name: Tenure, dtype: float64

In [14]:
# Show the distribution of the tenure of people above the 80th percentile
total_charges_above80.Tenure.describe()

count    374.000000
mean      50.133690
std       12.334841
min       27.000000
25%       40.000000
50%       49.500000
75%       60.000000
max       72.000000
Name: Tenure, dtype: float64

The average LTV of the 80% of those who unsubscribed is \\$ 711, and their tenure is near 10 months. On the hand, the average LTV of the 20 % of those who unsubscribed is $ 4811, and their tenure is near 50 months.  

## 2. What kinds of services they subscribed when they were still a customer?

### PhoneService

In [15]:
# under 80
pd.read_sql("select count(1) from churn_all where TotalCharges <= 2840.41 and Churn = 'Yes' group by PhoneService", conn)

,count(1)
0,75
1,659


In [16]:
df["Gender"].value_counts() # example

Male      3555
Female    3488
Name: Gender, dtype: int64

In [17]:
# under 80
total_charges_under80.PhoneService.value_counts()

Yes    1334
No      161
Name: PhoneService, dtype: int64

In [18]:
# Example of shape
df.shape

(7043, 25)

In [19]:
# Under 80 (percentage)
total_charges_under80.PhoneService.value_counts() / total_charges_under80.shape[0]

Yes    0.892308
No     0.107692
Name: PhoneService, dtype: float64

In [20]:
# Above 80 (percentage)
total_charges_above80.PhoneService.value_counts() / total_charges_above80.shape[0]

Yes    0.975936
No     0.024064
Name: PhoneService, dtype: float64

For the top 20% (high LTV) only 2% of them didn't use the phone service. On the other hand, lower 80% (LTV) had around 11% of the people that didn't use the phone service. That is around 5 times more as the top 20%. Apparently if you are not subscribed to the phone service you are more likely to be in the bottom 80% of the LTV. We as a company can try to sell this service to more people.

### Multiple Lines

In [21]:
total_charges_under80_use_phone = total_charges_under80.query("PhoneService == 'Yes'")

total_charges_under80_use_phone.MultipleLines.value_counts() / total_charges_under80_use_phone.shape[0]

No     0.590705
Yes    0.409295
Name: MultipleLines, dtype: float64

In [22]:
total_charges_above80_use_phone = total_charges_above80.query("PhoneService == 'Yes'")

total_charges_above80_use_phone.MultipleLines.value_counts() / total_charges_above80_use_phone.shape[0]

Yes    0.832877
No     0.167123
Name: MultipleLines, dtype: float64

For the top 20% of LTV who used the phone service 84% of them used multiple lines, which is 2 times as much as the proportion of people in the low 80% LTV that is around 40%. Having multiple lines is going to get you closer to be in the top 20% of the LTV.

### InternetService

In [23]:
# Under 80 (percentage)
total_charges_under80.InternetService.value_counts() / total_charges_under80.shape[0]

Fiber optic    0.640803
DSL            0.283612
No             0.075585
Name: InternetService, dtype: float64

In [24]:
# Above 80 (percentage)
total_charges_above80.InternetService.value_counts() / total_charges_above80.shape[0]

Fiber optic    0.906417
DSL            0.093583
Name: InternetService, dtype: float64

All the people in the high 20% LTV used internet service. On the other hand 8% of the people in the low 80th percentile of the LTV did not have internet connection. Also, in the top 20% (high LTV) 90% of them had fiber optic, in comparison to only 64% in the low LTV. So as a company if we want to increase the LTV of customers, we should recommend using internet and Fiber Optic connection.

### Other services

In [25]:
import numpy as np

In [26]:
total_charges_under80_use_internet = total_charges_under80.query('InternetService!="No"')
proportion_internet_sub_service_under80 = np.array([total_charges_under80_use_internet.query('OnlineSecurity=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('TechSupport=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('OnlineBackup=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('DeviceProtection=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('StreamingTV=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('StreamingMovies=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0]])

In [27]:
proportion_internet_sub_service_under80

array([0.1316932 , 0.13603473, 0.22720695, 0.23516643, 0.37698987,
       0.3813314 ])

In [28]:
total_charges_above80_use_internet = total_charges_above80.query('InternetService!="No"')
proportion_internet_sub_service_above80 = np.array([total_charges_above80_use_internet.query('OnlineSecurity=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('TechSupport=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('OnlineBackup=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('DeviceProtection=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('StreamingTV=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('StreamingMovies=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0]])

In [29]:
proportion_internet_sub_service_above80

array([0.30213904, 0.32620321, 0.55882353, 0.58823529, 0.78342246,
       0.77807487])

In the top 20% LTV, both streaming movies and streaming tv percentages are around 80%, in comparison in the low 80% LTV they are around 40%. Device protection and online backup for the top 20% LTV are around 55%, in comparison to only around 22% in the low LTV. As a company we would like to sell to customers streaming TV and movies and also online backup and device protection if we want to increase LTV.

## 3. For those who churned what is the proportion of each kind of contract?

In [30]:
# Under 80 (percentage)
total_charges_under80.Contract.value_counts() / total_charges_under80.shape[0]

Month-to-month    0.959197
One year          0.036789
Two year          0.004013
Name: Contract, dtype: float64

In [31]:
# Above 80 (percentage)
total_charges_above80.Contract.value_counts() / total_charges_above80.shape[0]

Month-to-month    0.590909
One year          0.296791
Two year          0.112299
Name: Contract, dtype: float64

## 4. For those who did not churn what is the proportion of each kind of contract?

In [32]:
## Extract those who unsubscribed the services (Churn = Yes)

paying_df = pd.read_sql("select * from churn_all where Churn = 'No'", conn)

In [33]:
paying_df['TotalCharges'] = paying_df['TotalCharges'].astype(float)
paying_df.TotalCharges.describe()

count    5174.000000
mean     2549.911442
std      2329.954215
min         0.000000
25%       572.900000
50%      1679.525000
75%      4262.850000
max      8672.450000
Name: TotalCharges, dtype: float64

In [34]:
# Find the 80th percentile of the data in TotalCharges
paying_df.TotalCharges.quantile(0.8)

4890.900000000001

In [35]:
# Divide the data by the 80th percentile of the data from the TotalCharges variable
paying_total_charges_under80 = paying_df.query("TotalCharges <= 4895.8")
paying_total_charges_above80 = paying_df.query("TotalCharges > 4895.8")

In [36]:
# Under 80 (percentage)
paying_total_charges_under80.Contract.value_counts() / paying_total_charges_under80.shape[0]

Month-to-month    0.499396
Two year          0.255011
One year          0.245593
Name: Contract, dtype: float64

In [37]:
# Above 80 (percentage)
paying_total_charges_above80.Contract.value_counts() / paying_total_charges_above80.shape[0]

Two year          0.572120
One year          0.280736
Month-to-month    0.147144
Name: Contract, dtype: float64

- If we want to stop Churning we would like to sell contracts to people for at least one year, and much better for 2 years (Conjecture).
- We have 57% of the people that stayed in the company in the top 20% LTV with 2 year contracts, and only 11% of the people in the top 20% had a 2 year contract and left the company.
- 95% of the people that left the company and are in the low 80% have a month-to-month contract, in contrast to only 50% of the people in the low 80% that stayed in the company. We can say that having a month-to-month contract is one of the reasons that people are leaving the company (conjecture).